# Inference code 

## inference on train set, and also test set
## evaluate metrics

In [ ]:
#first fix the device
# inference is faster with GPU
# obligatory device set up
# Automatically select device: GPU 0 if available, else CPU
device = torch.device("cuda:0" if torch.cuda.is_available() and torch.cuda.device_count() >= 1 else "cpu")
print(f"Using devic")


In [ ]:
# first the model



from transformers import pipeline

#infer_pipe = pipeline("text-generation", model="distilled-student-peft-adapter", 
#                      tokenizer="distilled-student-peft-adapter", device=0)

# need to use peft load  
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
adapter_path = "/kaggle/input/savedmodelsummarization/pytorch/default/1"   # change as needed

tokenizer = AutoTokenizer.from_pretrained(adapter_path)
base_model = AutoModelForCausalLM.from_pretrained(model_id)

# IMPORTANT: Use is_trainable=True for proper adapter loading
model = PeftModel.from_pretrained(base_model, adapter_path, is_trainable=True).to(device)

print("Trainable params after load:")
model.print_trainable_parameters()
print("Compare with previous data to check match...")

### sample inference
### check reasonableness before batch inference

In [ ]:
import torch
# Inference example

prompt = f"""Assessment: You have typhoiditis
Plan: Take paracetamol and acetenomycin
Rewrite the above for a patient with no medical background."""


inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.no_grad():
    gen_output = model.generate(
        **inputs,
        max_new_tokens=256,
        do_sample=True,
        top_p=0.95,
        temperature=0.7,
        eos_token_id=tokenizer.eos_token_id
    )


# Full output includes the prompt + generated text
full_output = tokenizer.decode(gen_output[0], skip_special_tokens=True)

# Extract only the generated part (remove prompt)  - 
generated_text = full_output[len(prompt):].strip()

print("Prompt: ", prompt)
print("\nGenerated summary:")
print(generated_text)
#print("\n\n")
#print(full_output)

## Above code can be reused with any Assessment and Plan text
